In [2]:
from ExpGSMM.model.COBRAmodel import *
from ExpGSMM import DATA_PATH
from cobra.flux_analysis import flux_variability_analysis as fva

In [3]:
model = MyModel(join(DATA_PATH, 'models/model_plutheri.xml'), 'e_Biomass__in')

Loading



2686093 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


Reactions: 2419
Metabolites: 2133
Genes: 1232
Model loaded


In [3]:
model = MyModel(r"C:\Users\Bisbii\PythonProjects\gap_filling_dl\tests\data\draft_synechocystis\model_synechocystis.xml", 'e_Biomass__cytop')

Loading



1148 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


Reactions: 1436
Metabolites: 1455
Genes: 621
Model loaded


In [6]:
model.test_reaction("R04457__cytop")

R07281__cytop not found


AttributeError: 'NoneType' object has no attribute 'reactants'

In [7]:
m = cobra.io.read_sbml_model(r"C:\Users\Bisbii\PythonProjects\gap_filling_dl\tests\data\draft_synechocystis\universal_model.xml")

1314884 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


In [8]:
for met in m.metabolites:
    if len(met.reactions) <=1:
        print(met.id, met.name)

C02027__in Deoxylimonate
C19755__in Patchoulol
C21849__in 4-O-(2-N-Acetyl-beta-D-glucosaminyl-D-ribitylphospho)n-di[(2R)-1-glycerophospho]-N-acetyl-beta-D-mannosaminyl-(1->4)-N-acetyl-alpha-D-glucosaminyl-diphospho-ditrans,octacis-undecaprenol
C03673__in 3-(ADP)-2-phosphoglycerate
C17236__in 3-Phenylpropionaldoxim
C17139__in dTDP-3,4-dioxo-2,6-dideoxy-L-glucose
C02462__in Sabinene hydrate
C21293__in Validamycin B
C12645__in Pelargonidin 3-O-rutinoside 5-O-beta-D-glucoside
C10552__in 4'-Demethyldeoxypodophyllotoxin
C21802__in (+)-Didemethylpinoresinol
C11531__in 6-O-(beta-D-Xylopyranosyl)-beta-D-glucopyranoside
C21617__in 2-Dehydro-D-erythronate
C21390__in Butane
C04740__in alpha-N-Acetylneuraminyl-2,6-beta-galactosyl-1,4-beta-D-glucosylceramide
C21811__in Wall teichoic acid
C16266__in 3-Chloro-2-hydroxymuconic semialdehyde
C02453__in Phenolic steroid
C10126__in Anatabine
C05622__in Malonylapiin
C22360__in S-methyl-1-thio-D-ribulose 5-phosphate
C01048__in Polyprenyl phosphate
C04875__in

In [12]:
model.reactions.R04457__cytop

Reaction identifier,R04457__cytop
Name,"5-Amino-6-(1-D-ribitylamino)uracil + L-3,4-Dihydroxybutan-2-one 4-phosphate =>..."
Memory address,0x206fae44f10
Stoichiometry,"C04732__cytop + C15556__cytop --> 2.0 C00001__cytop + C00009__cytop + C00080__cytop + C04332__cytop 5-Amino-6-(1-D-ribitylamino)uracil + L-3,4-Dihydroxybutan-2-one 4-phosphate --> 2.0 H2O + Orthophosphate + H+ + 6,7-Dimethyl-8-(D-ribityl)lumazine"
GPR,WP_010872197_1
Lower bound,0.0
Upper bound,1000.0


In [62]:
model.metabolites.C00169__in.summary(fva= 0.0)

Percent,Flux,Range,Reaction,Definition
,0,[0; 1000],R00149__in,C00001__in + 2.0 C00002__in + C00011__in + C00014__in --> 2.0 C00008__in + C00009__in + C00169__in
,0,[0; 1000],R00575__in,C00001__in + 2.0 C00002__in + C00064__in + C00288__in --> 2.0 C00008__in + C00009__in + C00025__in + C00169__in
,0,[0; 1000],R01395__in,C00002__in + C01563__in --> C00008__in + C00169__in
,0,[0; 1000],R07641__in,2.0 C00002__in + C00014__in + C00288__in --> 2.0 C00008__in + C00009__in + C00169__in
Percent,Flux,Range,Reaction,Definition
,0,[-1000; 0],R01398__in,C00077__in + C00169__in --> C00009__in + C00327__in


In [38]:
fva(model, "R00149__in")

,minimum,maximum
R00149__in,0.0,1000.0


In [108]:
model.create_sink("C05752__in")
model.create_sink("C22158__in")
model.create_sink("C22155__in")
model.create_sink("C15811__in")

Reaction identifier,Sk_C15811__in
Name,
Memory address,0x1d6c8c5b400
Stoichiometry,<=> C15811__in <=> [Enzyme]-cysteine
GPR,
Lower bound,-10000
Upper bound,10000


In [118]:
model.test_reaction("R06284__in")

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


,Flux
Chlorophyllide,0.000000
Phytyl diphosphate,219.512195


In [120]:
model.test_reaction("e_Pigment__in")

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


,Flux
Diadinoxanthin,103.448276
beta-Carotene,107.142857
chlorophyll c1,105.882353
chlorophyll c2,105.882353
Chlorophyll a,72.000000
Fucoxanthin,97.826087


In [121]:
model.test_reaction("e_Biomass__in")

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


,Flux
e-Pigment,16.129032
e-Protein,50.260772
e-Cofactor,1000.000000
e-Lipid,0.000000
e-DNA,49.445940
e-RNA,56.741225
e-Carbohydrate,171.126561
